### 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

### 2. Helper functions

In [2]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    
    return state

### 3. Initialize the environment

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
#print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = len(env.observation) + 8
nb_actions = 200
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

27 9

Ram information received from DASSAT will has 35 dimensions.
There are 200 possible actions at each step.
Discrete? False


### 4. Define the network

In [4]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc2(x)

### 5. Define the Replay Buffer

In [5]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [6]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [7]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        # return the action index with the largest value
        else:
            return random.choice(np.arange(self.action_size))
            #return one action randomly with possibility eps

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # detach for diemnsion correctness
        # GET largest value of self.qnetwork_target(next_states)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # 0 if done

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        #Why need to gather? actions are indexs?

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
        #固定用法, change learning rate

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())

### DQN for nitrogen management

In [8]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [9]:
def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0.01, eps_decay=0.997, solved=495, optimize = True):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    action_list=[]
    action_num_list=[]
    action_not_zero=0
    for i_episode in range(1, n_episodes+1):
        # Reset env and score at the beginning of episode
        action_not_zero=0
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        for t in range(max_t):
            action1 = agent.act(state, eps) if optimize else 100
            if action1 != 0:
                action_not_zero=action_not_zero+1
            if i_episode == 1000:
                action_list.append(action1)
            #action_n++
            # for debug
            action = {
                    'anfer': 0,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': action1/4,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
            next_state, reward, done, _ = env.step(action)      # send the action to the environment\
            if done:
                print('episode is %d .' % i_episode)
                print('action_not_zero is %d .' % action_not_zero)
                print('score is %d .' % score)
                print('')
                if i_episode > 900:
                    action_num_list.append(action_not_zero)
                break
            if action_not_zero > 5:
                if action1 > 0:
                    #print('satisfied')
                    `
            next_state = dict2array(next_state)
            #print('after reward is %d .' % reward)
            agent.step(state, action['anfer'], reward, next_state, done)
            state = next_state
            score += reward
            #append

        if i_episode == 1000:
            print('action_not_zero is %d .' % action_not_zero)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        list_eps.append(eps)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
            
        
        if i_episode % 100 == 0:
            print('\rEpisode {}/{} \tAverage Score: {:.2f}'.format(i_episode, n_episodes, np.mean(scores_window)))
            print('Epsilon: {}'.format(eps))
            
        if np.mean(scores_window)>solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break

    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds".format(time_elapsed))
    
    return scores, list_eps, action_list, action_num_list

In [10]:
scores, list_eps, action_list, action_num_list= dqn(n_episodes=1000)

episode is 1 .
action_not_zero is 153 .
score is -1111111111111111145812933925308117198078195805572231318613161213538824647224794933586283510323636084819196517350018863265277425295199695012964073472 .

episode is 2 .
action_not_zero is 154 .
score is -11111111111111111813031060100545474006818973626037027225995557783492768094069335654534362502356440597308689154830054629118849256011654144238332936192 .

episode is 3 .
action_not_zero is 152 .
score is -111111111111111101272478860750900393831431496670421355366418159549962903904177506408446073565360617890042502435131911459049929914782824722220449792 .

episode is 4 .
action_not_zero is 153 .
score is -1111111111111111145812933925308117198078195805572231318613161213538824647224794933586283510323636084819196517350018863265277425295199695012964073472 .

episode is 5 .
action_not_zero is 152 .
score is -1111111111111111012724788607509003938314314966704213553664181595499629039041775064084460735653606178900425024351319114590499299147828247222204

episode is 44 .
action_not_zero is 139 .
score is -11111111111111110939169404565979554330244027043994518924125932702702196497709696572463638339410976361592440582158260891065877597782016 .

episode is 45 .
action_not_zero is 139 .
score is -111111111111111106869965649090548873716581704030753905716156013717233378228406187852910190018288284485411365508948007899034584598183936 .

episode is 46 .
action_not_zero is 141 .
score is -1111111111111111068699656490905488737165817040307539057161560137172333782284061878529101900182882844854113655089480078990345845981839360 .

episode is 47 .
action_not_zero is 132 .
score is -1111111111111111044331593498606016334485940765873994929603993650265045969447172493788112504995901879910226275640194701874692096 .

episode is 48 .
action_not_zero is 133 .
score is -11111111111111109541475583200908203835298858491634530783777837721855629342871991869512859405949376509041847376650765054483890176 .

episode is 49 .
action_not_zero is 136 .
score is -11111111111111

episode is 91 .
action_not_zero is 120 .
score is -1111111111111111143588592399289630612884822573784661074252310581681568343139802214807990705456829964247959596957696 .

episode is 92 .
action_not_zero is 113 .
score is -1111111111111111134402179404957904047584358126547642018871147772639870297391002174916334775439309473719517184 .

episode is 93 .
action_not_zero is 124 .
score is -11111111111111111486192690662905580389415776961551902145646002727359569469190746095508826036153945876620399041107197952 .

episode is 94 .
action_not_zero is 112 .
score is -11111111111111111669947550263096814278794891410976925956946420712948503075692493645864355716526967726538752 .

episode is 95 .
action_not_zero is 117 .
score is -1111111111111111018633350797916390792776052762865862369994096023817381301237868157208136747289357787904677511168 .

episode is 96 .
action_not_zero is 116 .
score is -111111111111111111875774310670905090504269523956361774199587050928547989250544352387468639325688357023102533632 

episode is 141 .
action_not_zero is 108 .
score is -1111111111111111057981501605284449467419700347314564445674653816516230644619545629605873826505380855808 .

episode is 142 .
action_not_zero is 103 .
score is -111111111111111117599863680799749052577701118684333802420086928893370656066714952124430915272179712 .

episode is 143 .
action_not_zero is 106 .
score is -111111111111111112014690615650777973684248136566716945823167266618469532467754274125081095307205279744 .

episode is 144 .
action_not_zero is 101 .
score is -111111111111111116651294885767654779667807609493162461286099214512443155455478423931606557261824 .

episode is 145 .
action_not_zero is 93 .
score is -11111111111111109874920465169055439424001067332574564517448701497577173312074515264569344 .

episode is 146 .
action_not_zero is 104 .
score is -1111111111111110993873428161835390127077457422138440526475228127795626443309736108222032414733500416 .

episode is 147 .
action_not_zero is 105 .
score is -111111111111111099387342

episode is 198 .
action_not_zero is 98 .
score is -111111111111111112714363851698904135266394119197773594274919735880663978113920567662560149504 .

episode is 199 .
action_not_zero is 82 .
score is -11111111111111111098355479645952903587745526339464277786242012666733776601088 .

episode is 200 .
action_not_zero is 78 .
score is -11111111111111110922596631055125842200343422489649618135376060221740810240 .

Episode 200/1000 	Average Score: -12346889136701738198367636902686438998917950212451529151066144464412440156131259861962670205632941673666223777251328.00
Epsilon: 0.5483169386261801
episode is 201 .
action_not_zero is 94 .
score is -111111111111111105816592910804091712573200676297419708484422602478247565607169957816172544 .

episode is 202 .
action_not_zero is 90 .
score is -1111111111111111271153821369997715605968572672843202547485448905960604748387197124608 .

episode is 203 .
action_not_zero is 94 .
score is -1111111111111111058165929108040917125732006762974197084844226024782475656

...

episode is 501 .
action_not_zero is 30 .
score is -11111111111111113072181248 .

episode is 502 .
action_not_zero is 45 .
score is -1111111111111111010540023201374943576064 .

episode is 503 .
action_not_zero is 36 .
score is -1111111111111111195755133534208 .

episode is 504 .
action_not_zero is 35 .
score is -1111111111111111195755133534208 .

episode is 505 .
action_not_zero is 34 .
score is -11111111111111109670567149568 .

episode is 506 .
action_not_zero is 39 .
score is -11111111111111109993981897808543744 .

episode is 507 .
action_not_zero is 34 .
score is -111111111111111105501764517888 .

episode is 508 .
action_not_zero is 32 .
score is -111111111111111122131877888 .

episode is 509 .
action_not_zero is 29 .
score is -1111111111111111092469760 .

episode is 510 .
action_not_zero is 29 .
score is -1111111111111111092469760 .

episode is 511 .
action_not_zero is 28 .
score is -111111111111111109246976 .

episode is 512 .
action_not_zero is 26 .
score is -111111111111111108198

episode is 601 .
action_not_zero is 16 .
score is -111111111527 .

episode is 602 .
action_not_zero is 30 .
score is -11111111111111113072181248 .

episode is 603 .
action_not_zero is 25 .
score is -111111111111111114752 .

episode is 604 .
action_not_zero is 17 .
score is -1111111111559 .

episode is 605 .
action_not_zero is 28 .
score is -111111111111111109246976 .

episode is 606 .
action_not_zero is 26 .
score is -1111111111111111081984 .

episode is 607 .
action_not_zero is 15 .
score is -11111111537 .

episode is 608 .
action_not_zero is 24 .
score is -1111111111111111552 .

episode is 609 .
action_not_zero is 26 .
score is -111111111111111114752 .

episode is 610 .
action_not_zero is 19 .
score is -111111111111531 .

episode is 611 .
action_not_zero is 14 .
score is -1111111508 .

episode is 612 .
action_not_zero is 27 .
score is -11111111111111111344128 .

episode is 613 .
action_not_zero is 18 .
score is -11111111111525 .

episode is 614 .
action_not_zero is 25 .
score is -111

episode is 713 .
action_not_zero is 15 .
score is -11111111527 .

episode is 714 .
action_not_zero is 22 .
score is -111111111111111520 .

episode is 715 .
action_not_zero is 17 .
score is -1111111111539 .

episode is 716 .
action_not_zero is 21 .
score is -11111111111111542 .

episode is 717 .
action_not_zero is 18 .
score is -11111111111522 .

episode is 718 .
action_not_zero is 14 .
score is -1111111539 .

episode is 719 .
action_not_zero is 18 .
score is -11111111111527 .

episode is 720 .
action_not_zero is 16 .
score is -111111111526 .

episode is 721 .
action_not_zero is 14 .
score is -1111111514 .

episode is 722 .
action_not_zero is 16 .
score is -11111111591 .

episode is 723 .
action_not_zero is 20 .
score is -1111111111111557 .

episode is 724 .
action_not_zero is 23 .
score is -1111111111111111424 .

episode is 725 .
action_not_zero is 18 .
score is -11111111111567 .

episode is 726 .
action_not_zero is 12 .
score is -11111520 .

episode is 727 .
action_not_zero is 15 .
sc

episode is 836 .
action_not_zero is 14 .
score is -1111111538 .

episode is 837 .
action_not_zero is 13 .
score is -11111513 .

episode is 838 .
action_not_zero is 17 .
score is -1111111111528 .

episode is 839 .
action_not_zero is 9 .
score is -11576 .

episode is 840 .
action_not_zero is 12 .
score is -11111507 .

episode is 841 .
action_not_zero is 7 .
score is -502 .

episode is 842 .
action_not_zero is 16 .
score is -111111111521 .

episode is 843 .
action_not_zero is 10 .
score is -11571 .

episode is 844 .
action_not_zero is 15 .
score is -11111111527 .

episode is 845 .
action_not_zero is 9 .
score is -1545 .

episode is 846 .
action_not_zero is 13 .
score is -111111516 .

episode is 847 .
action_not_zero is 10 .
score is -111517 .

episode is 848 .
action_not_zero is 11 .
score is -1111509 .

episode is 849 .
action_not_zero is 11 .
score is -1111562 .

episode is 850 .
action_not_zero is 14 .
score is -111111520 .

episode is 851 .
action_not_zero is 9 .
score is -11500 .

ep

episode is 970 .
action_not_zero is 9 .
score is -11557 .

episode is 971 .
action_not_zero is 10 .
score is -111513 .

episode is 972 .
action_not_zero is 8 .
score is -1502 .

episode is 973 .
action_not_zero is 8 .
score is -1509 .

episode is 974 .
action_not_zero is 10 .
score is -111496 .

episode is 975 .
action_not_zero is 9 .
score is -11515 .

episode is 976 .
action_not_zero is 7 .
score is -547 .

episode is 977 .
action_not_zero is 9 .
score is -11505 .

episode is 978 .
action_not_zero is 8 .
score is -1503 .

episode is 979 .
action_not_zero is 8 .
score is -1555 .

episode is 980 .
action_not_zero is 8 .
score is -517 .

episode is 981 .
action_not_zero is 6 .
score is -390 .

episode is 982 .
action_not_zero is 11 .
score is -1111580 .

episode is 983 .
action_not_zero is 8 .
score is -1549 .

episode is 984 .
action_not_zero is 7 .
score is -506 .

episode is 985 .
action_not_zero is 12 .
score is -1111550 .

episode is 986 .
action_not_zero is 6 .
score is -396 .

ep

### Given a constant input: reward = 19847

In [16]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(scores)
plt.yscale('symlog')
plt.title("Water Management")
plt.xlabel("Traing Steps")
plt.ylabel("Rewards")
plt.savefig("reward_0 nitrogen with exponential penalty learning_false.png")

In [17]:
plt.figure()
print(action_list)
plt.plot(action_list)
plt.title("W action of last episode")
plt.xlabel("Days")
plt.ylabel("Nitrogen input")
plt.savefig("action_0 nitrogen with linear penalty learning_false.png")

[0, 0, 0, 0, 0, 0, 193, 0, 0, 0, 0, 0, 0, 0, 159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 42, 0, 0, 0, 50, 0, 0, 0, 0, 0, 0, 136, 0, 44, 0, 0, 0, 0, 0, 0, 182, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0]


In [18]:
print(np.mean(action_num_list))

9.05
